# Car Counter Project 

In [1]:
import cv2 
import cvzone
import math
from sort import *

#### Use a pre pretrained Model 

In [2]:
from ultralytics import YOLO
model =YOLO("../Yolo-Weights/yolov8s.pt")

#### Import all classes 

In [4]:
all_classes = []
with open("../YOLO Basics/80Classes.txt","r") as f:
    all_classes = f.read().splitlines()
    # OR : 
    # all_classes = [line.strip() for line in f.readlines()]

#### Test the mask Operation to select Corresponding Region 

In [6]:
mask = cv2.imread("./Mask.png")
h,w = mask.shape[:2]
bycycle =cv2.imread("../YOLO Basics/images/3bikes.jpg")
bycycle = cv2.resize(bycycle,(w,h))

masked_image =cv2.bitwise_and(mask,bycycle)
# cv2.imshow("Mask",masked_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [9]:
cap = cv2.VideoCapture("../videos/cars.mp4") # Open The Video = '0'
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280) # Width is equal to 1280 pixels 
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720)

#create instance of SORT
tracker = Sort(max_age=20 ,min_hits=3,iou_threshold=0.3) 
totalCount = set()

while True:
    success,frame = cap.read()
    img_region = cv2.bitwise_and(frame,mask)
    results =model.predict(img_region, stream=True)

    carLabel =cv2.imread("graphics.png",cv2.IMREAD_UNCHANGED)
    frame =cvzone.overlayPNG(frame,carLabel,(0,0))

    detections = np.empty((0,5))
    cv2.line(frame,(400,297),(673,297),(0,0,255),3)

    for r in results :
          boxes = r.boxes # Len Boxes = Number Of Objects
          centers = []
          for box in boxes:
              #print(box)
              x1,y1,x2,y2 = box.xyxy[0]
              x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
              w, h = x2-x1, y2-y1
              
              # the Class Name 
              cls = int(box.cls[0])
              class_detected = all_classes[cls]
              #The Confidence Sore 
              conf = math.ceil(box.conf[0]*100)/100 
              
              # Display only the Permitted classes
              if class_detected in ["bicycle","car"] and conf >0.5 :
                
                # Track the Object Detected Only    
                currentArray = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections, currentArray))
                
                # # Display the Bounding Box
                # cvzone.putTextRect(frame,f"{class_detected} {conf} ",(x1,y1+30),colorR=(0,255,0),colorB=(0,0,0),scale=0.8,thickness=1,offset=5)
                cvzone.cornerRect(frame,(x1,y1,w,h),colorR=(255,255,255),rt=1,l=10)
                cx ,cy = x1+w//2,y1+h//2
                #cv2.circle(frame,(cx,cy),5,(0,255,0),cv2.FILLED)
                centers.append([cx,cy])
                
    resultsTrackers = tracker.update(detections)
    
    for i,results in enumerate(resultsTrackers):
         
      x1,y1,x2,y2,id = results
      x1,y1,x2,y2,id = int(x1),int(y1),int(x2),int(y2),int(id)
      #cvzone.cornerRect(frame,(x1,y1,w,h),colorR=(255,0,0),rt=1,l=10)
      #cvzone.putTextRect(frame,str(id),(x1,y1),colorR=(255,0,0),colorB=(255,255,255),scale=1,thickness=2,offset=5) 
      cx ,cy =centers[i]
      cv2.circle(frame,(cx,cy),5,(0,255,0),cv2.FILLED)
      if 297-16 <= cy <= 297+16 and 400<= cx <= 673:
        if id not in totalCount:
          totalCount.add(id)
          cv2.line(frame,(400,297),(673,297),(0,255,0),3)
        
    
    cv2.putText(frame,f"{len(totalCount)}",(200,90),cv2.FONT_HERSHEY_PLAIN,5,(0,255,0),4)
    

    cv2.imshow("frame",frame)

    if cv2.waitKey(100) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            cap.release()
            break
        


0: 384x640 4 cars, 1 truck, 92.9ms
Speed: 3.7ms preprocess, 92.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 90.0ms
Speed: 5.8ms preprocess, 90.0ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 88.1ms
Speed: 4.8ms preprocess, 88.1ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 88.2ms
Speed: 3.1ms preprocess, 88.2ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 87.0ms
Speed: 3.3ms preprocess, 87.0ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 87.1ms
Speed: 4.0ms preprocess, 87.1ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 87.4ms
Speed: 4.0ms preprocess, 87.4ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 87.1ms
Speed: 3.6ms preprocess, 87.1ms inference, 4.3ms postprocess per image at shape (1, 3, 